In [48]:
from pyspark.sql import SparkSession
from operator import add
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.119:7077") \
        .appName("tim-wywiol-intro")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()
# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

In [92]:
# Loading Data in SPARK RDD from HDFS
rdd_lines_en = spark_context.textFile("hdfs://192.168.2.119:9000/europarl/europarl-v7.de-en.en")
rdd_lines_de = spark_context.textFile("hdfs://192.168.2.119:9000/europarl/europarl-v7.de-en.de")


In [93]:
## A1 
## Count lines
print(rdd_lines_en.first())
print(rdd_lines_en.getNumPartitions())
print(rdd_lines_en.count())
print(rdd_lines_de.first())
print(rdd_lines_de.getNumPartitions())
print(rdd_lines_de.count())

Resumption of the session
3


1920209
Wiederaufnahme der Sitzungsperiode
3


1920209


In [94]:
## A2
#LowerCase RDD Text
def TokLow(line):
    new_line = line.lower()
    return new_line.split()
rdd_lines_en_low = rdd_lines_en.map(lambda x: TokLow(x))
rdd_lines_de_low = rdd_lines_de.map(lambda x: TokLow(x))

In [95]:
print(rdd_lines_en_low.take(2))
print(rdd_lines_en_low.count())
print(rdd_lines_de_low.take(2))
print(rdd_lines_de_low.count())

[['resumption', 'of', 'the', 'session'], ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.']]


1920209
[['wiederaufnahme', 'der', 'sitzungsperiode'], ['ich', 'erkläre', 'die', 'am', 'freitag,', 'dem', '17.', 'dezember', 'unterbrochene', 'sitzungsperiode', 'des', 'europäischen', 'parlaments', 'für', 'wiederaufgenommen,', 'wünsche', 'ihnen', 'nochmals', 'alles', 'gute', 'zum', 'jahreswechsel', 'und', 'hoffe,', 'daß', 'sie', 'schöne', 'ferien', 'hatten.']]


1920209


In [32]:
## A3
# Count 10 most frequent words
# Map
all_words_en = rdd_lines_en_low\
                                .flatMap(lambda w: w)\
                                .map(lambda l: (l,1))
print(all_words_en.take(20))


[('resumption', 1), ('of', 1), ('the', 1), ('session', 1), ('i', 1), ('declare', 1), ('resumed', 1), ('the', 1), ('session', 1), ('of', 1), ('the', 1), ('european', 1), ('parliament', 1), ('adjourned', 1), ('on', 1), ('friday', 1), ('17', 1), ('december', 1), ('1999,', 1), ('and', 1)]


In [37]:
from operator import add
all_words_counts_en = all_words_en.reduceByKey(add)
print(all_words_counts_en.takeOrdered(10, key=lambda x: -x[1]))

[('the', 3663317), ('of', 1737096), ('to', 1611846), ('and', 1345289), ('in', 1134117), ('that', 835926), ('a', 810820), ('is', 792599), ('for', 557377), ('we', 551272)]


In [38]:
## A3 Deutsch
all_words_de = rdd_lines_de_low\
                                .flatMap(lambda w: w)\
                                .map(lambda l: (l,1))
all_words_counts_de = all_words_de.reduceByKey(add)
print(all_words_counts_de.takeOrdered(10, key=lambda x: -x[1]))

[('die', 1980588), ('der', 1710400), ('und', 1338008), ('in', 781398), ('zu', 618887), ('den', 577690), ('wir', 489054), ('für', 478353), ('ich', 469069), ('das', 466167)]


In [96]:
#######  A4   #################################################################
#1 key line ((1, [w1, w2, w3]), (2, [w1,w2,w3])...
en_1  = rdd_lines_en_low.zipWithIndex()
#[(['resumption', 'of', 'the', 'session'], 0),
en_2 = en_1.map(lambda x: (x[1], x[0]))
print(en_2.take(3))

[(0, ['resumption', 'of', 'the', 'session']), (1, ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.']), (2, ['although,', 'as', 'you', 'will', 'have', 'seen,', 'the', 'dreaded', "'millennium", "bug'", 'failed', 'to', 'materialise,', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful.'])]


In [97]:
de_1  = rdd_lines_de_low.zipWithIndex()
de_2 = de_1.map(lambda x: (x[1], x[0]))

In [98]:
#4.3 Join RDDs
print(en_2.take(2))
print(de_2.take(2))
join_3 = en_2.join(de_2)
print(join_3.take(5))

[(0, ['resumption', 'of', 'the', 'session']), (1, ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.'])]
[(0, ['wiederaufnahme', 'der', 'sitzungsperiode']), (1, ['ich', 'erkläre', 'die', 'am', 'freitag,', 'dem', '17.', 'dezember', 'unterbrochene', 'sitzungsperiode', 'des', 'europäischen', 'parlaments', 'für', 'wiederaufgenommen,', 'wünsche', 'ihnen', 'nochmals', 'alles', 'gute', 'zum', 'jahreswechsel', 'und', 'hoffe,', 'daß', 'sie', 'schöne', 'ferien', 'hatten.'])]


[(696, (['if', 'we', 'do', 'not', 'wish', 'to', 'stand', 'accused', 'of', 'pursuing', 'a', 'cost-intensive', 'structural', 'policy', 'that', 'does', 'nothing', 'to', 'improve', 'the', 'unemployment', 'situation', 'in', 'the', 'long', 'term,', 'then', 'the', 'measures', 'drawn', 'up', 'so', 'far', 'must', 'be', 'analysed.'], ['wenn', 'wir', 'uns', 'nicht', 'den', 'vorwurf', 'gefallen', 'lassen', 'wollen,', 'eine', 'kostenintensive', 'strukturpolitik', 'zu', 'betreiben,', 'die', 'in', 'der', 'frage', 'der', 'beschäftigung', 'nichts', 'nachhaltig', 'bewegt,', 'müssen', 'die', 'bisherigen', 'maßnahmen', 'hinterfragt', 'werden.'])), (23736, (['with', 'regard', 'to', 'the', 'trans-european', 'networks', 'we', 'cannot', 'include', 'anything', 'which', 'is', 'not', 'supported', 'by', 'the', 'member', 'states.'], ['in', 'die', 'transeuropäischen', 'netze', 'können', 'wir', 'nichts', 'aufnehmen,', 'was', 'von', 'den', 'mitgliedstaaten', 'nicht', 'unterstützt', 'wird.'])), (24966, (['of', 'course

In [85]:
####### EXAMPLE
x = spark_context.parallelize([(2, ["Hi","Baum"]), (1, ["Die","Sonne","lacht", "sehr"]), (3, []),(4, ["Test2"])])
y = spark_context.parallelize([(2, ["Hej","Tree"]), (1, ["The","Sun","shines"]), (3, ["Test"]), (4, [])])
join = x.join(y)
print(join.collect())

 

[(4, (['Test2'], [])), (1, (['Die', 'Sonne', 'lacht', 'sehr'], ['The', 'Sun', 'shines'])), (2, (['Hi', 'Baum'], ['Hej', 'Tree'])), (3, ([], ['Test']))]


In [86]:
print(join.map(lambda x: len(x[1][0])).collect())
result = join \
    .filter(lambda x: (len(x[1][0]) != 0)) \
    .filter(lambda x: (len(x[1][1]) != 0)) \
    .filter(lambda x: (len(x[1][1]) <= 5)) \
    .filter(lambda x: (len(x[1][1])) == (len(x[1][0]))) 
print(result.collect())

[1, 4, 2, 0]
[(2, (['Hi', 'Baum'], ['Hej', 'Tree']))]


In [99]:
## A4.4 Exclude
# Exclude missing corresponding sentences
en_4 = join_3 \
    .filter(lambda x: len(x[1][0]) != 0) \
    .filter(lambda x: len(x[1][1]) != 0) \
    .filter(lambda x: len(x[1][0]) < 10) \
    .filter(lambda x: len(x[1][1]) == len(x[1][0])) 
         
print(en_4.take(5))

[(40224, (['question', 'no', '7', 'by', '(h-0274/00):'], ['anfrage', 'nr.', '7', 'von', '(h-0274/00):'])), (98064, (['fatuzzo', '(ppe-de).'], ['fatuzzo', '(ppe-de).'])), (98772, (['it', 'is', 'now', 'or', 'never.'], ['es', 'gilt', 'jetzt', 'oder', 'nie.'])), (195702, (['but', 'in', 'reality', 'they', 'do', 'not.'], ['aber', 'in', 'wirklichkeit', 'funktioniert', 'das', 'nicht.'])), (369954, (['in', 'my', 'opinion', 'the', 'report', 'before', 'us', 'is', 'excellent.'], ['meiner', 'meinung', 'nach', 'liegt', 'uns', 'ein', 'ausgezeichneter', 'bericht', 'vor.']))]


In [115]:
## A4.7
en_7 = en_4.flatMap(lambda x: tuple(zip(x[1][0], x[1][1])))
print(en_7.take(4))

[('we', 'wir'), ('have', 'haben'), ('a', 'eine'), ('new', 'neue')]


In [118]:
from operator import add
en_8 = en_7 \
    .map(lambda x: (x,1)) \
    .reduceByKey(add)


[(('have', 'haben'), 964), (('be', 'wir'), 54), (('our', 'uns'), 7), (('report', 'cederschiöld'), 5), (('(b5-0227/2000)', '(b5-0227/2000)'), 1)]


In [120]:
print(en_8.takeOrdered(20, key=lambda x: -x[1]))

[(('is', 'ist'), 11734), (('the', 'die'), 9815), (('debate', 'aussprache'), 4578), (('we', 'wir'), 4412), (('i', 'ich'), 4391), (('closed.', 'geschlossen.'), 4329), (('the', 'der'), 3085), (('that', 'das'), 2802), (('(applause)', '(beifall)'), 2746), (('.', '.'), 2644), (('and', 'und'), 2490), (('this', 'das'), 1894), (('mr', 'herr'), 1846), (('not', 'nicht'), 1703), (('thank', 'vielen'), 1593), (('a', 'eine'), 1454), (('are', 'sind'), 1449), (('a', 'ein'), 1440), (('by', 'von'), 1434), (('no', 'nr.'), 1407)]


In [47]:
# release the cores for another application!
spark_context.stop()